In [30]:
#!/usr/bin/env python
# coding: utf-8

# # Pronóstico de Casos de Dengue por Barrio usando GRU - Versión Optimizada
# ## Red Neuronal Recurrente para Series Temporales Multivariadas con MSE Minimizado

# ### 1. Configuración e Importación de Librerías

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar dispositivo de cómputo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo de cómputo: {device}")

# Establecer semilla para reproducibilidad
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

Dispositivo de cómputo: cuda


In [31]:
# ### 2. Carga y Preparación de Datos

# Cargar datos desde el archivo Parquet
df = pd.read_parquet('../../Datos/df_train.parquet')
print(f"Dimensiones del DataFrame: {df.shape}")
print("\nPrimeras filas:")
print(df.head())
print("\nInformación del DataFrame:")
print(df.info())

# Eliminar la columna 'lluvia_min' de df si existe
if 'lluvia_min' in df.columns:
    df = df.drop(columns=['lluvia_min'])

# Eliminar la columna 'id' de df si existe
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Dejar solo las columnas requeridas
#df = df[['id_bar','anio', 'semana', 'ESTRATO', 'dengue', 'lluvia_mean', 'temperatura_mean']]
df = df[['id_bar','anio', 'semana', 'dengue', 'lluvia_mean', 'temperatura_mean']]

# Eliminar registros con semana=53
df = df[df['semana'] != 53]

# Crear columna de fecha usando formato de semanas ISO
df['fecha'] = pd.to_datetime(df['anio'].astype(str) + df['semana'].astype(str) + '1', format='%G%V%u')

# Establecer fecha como índice y ordenar
df = df.set_index('fecha').sort_index()
print(f"\nRango temporal: {df.index.min()} - {df.index.max()}")

Dimensiones del DataFrame: (3680, 20)

Primeras filas:
          id  id_bar  anio  semana  ESTRATO  area_barrio  dengue  \
0  4_2015_01       4  2015       1      3.0        0.560     0.0   
1  5_2015_01       5  2015       1      3.0        0.842     0.0   
2  3_2015_01       3  2015       1      1.0        0.781     0.0   
3  8_2015_01       8  2015       1      2.0        0.394     0.0   
4  9_2015_01       9  2015       1      2.0        0.292     0.0   

   concentraciones  vivienda  equipesado  sumideros  maquina  lluvia_mean  \
0              0.0       0.0         0.0        0.0      0.0     0.000651   
1              0.0       0.0         0.0        0.0      0.0     0.000651   
2              0.0       0.0         0.0        0.0      0.0     0.000651   
3              0.0       0.0         0.0        0.0      0.0     0.000651   
4              0.0       0.0         0.0        0.0      0.0     0.000651   

   lluvia_var  lluvia_max  lluvia_min  temperatura_mean  temperatura_var 

In [32]:
df

,id_bar,anio,semana,dengue,lluvia_mean,temperatura_mean
fecha,,,,,,
2014-12-29,4,2015,1,0.0,0.000651,26.163889
2014-12-29,5,2015,1,0.0,0.000651,26.163889
2014-12-29,3,2015,1,0.0,0.000651,26.163889
2014-12-29,8,2015,1,0.0,0.000651,26.163889
2014-12-29,9,2015,1,0.0,0.000651,26.163889
...,...,...,...,...,...,...
2021-12-27,2,2021,52,0.0,0.142469,24.750446
2021-12-27,6,2021,52,0.0,0.142469,24.750446
2021-12-27,1,2021,52,1.0,0.142469,24.750446


In [33]:
# ### 3. Preprocesamiento e Ingeniería de Características Mejorada

# División de datos según el año
train_data = df[df['anio'] < 2021]
val_data = df[df['anio'] == 2021]
print(f"\nTamaño conjunto entrenamiento: {len(train_data)}")
print(f"Tamaño conjunto validación: {len(val_data)}")

# Identificar columnas numéricas y categóricas
categorical_cols = ['id_bar', 'ESTRATO']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['anio', 'semana']]
target_col = 'dengue'

print(f"\nColumnas categóricas: {categorical_cols}")
print(f"Columnas numéricas: {numerical_cols}")
print(f"Variable objetivo: {target_col}")

# ### MEJORA 1: Transformación logarítmica para estabilizar la varianza
# Aplicar transformación log1p a la variable objetivo para manejar valores sesgados
df['dengue_log'] = np.log1p(df['dengue'])
target_col_transformed = 'dengue_log'

# ### MEJORA 2: Ingeniería de características adicionales
# Agregar características temporales cíclicas
df['sin_semana'] = np.sin(2 * np.pi * df['semana'] / 52)
df['cos_semana'] = np.cos(2 * np.pi * df['semana'] / 52)

# Agregar lags de la variable objetivo por barrio
lag_features = []
for lag in [1, 2, 4, 8, 12]:  # Lags de 1, 2, 4, 8 y 12 semanas
    lag_col = f'dengue_lag_{lag}'
    df[lag_col] = df.groupby('id_bar')['dengue'].shift(lag)
    lag_features.append(lag_col)

# Agregar media móvil y desviación estándar móvil
for window in [4, 8, 12]:
    ma_col = f'dengue_ma_{window}'
    std_col = f'dengue_std_{window}'
    # Usar transform para mantener el índice original
    df[ma_col] = df.groupby('id_bar')['dengue'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    df[std_col] = df.groupby('id_bar')['dengue'].transform(lambda x: x.rolling(window=window, min_periods=1).std())
    lag_features.extend([ma_col, std_col])

# Rellenar valores faltantes en features de lag
df[lag_features] = df[lag_features].fillna(0)
# Para las desviaciones estándar, también rellenar NaN que pueden aparecer en ventanas pequeñas
std_cols = [col for col in lag_features if 'std' in col]
df[std_cols] = df[std_cols].fillna(0)

# Actualizar lista de columnas numéricas
numerical_cols.extend(['sin_semana', 'cos_semana', 'dengue_log'] + lag_features)
numerical_cols = list(set(numerical_cols))  # Eliminar duplicados

# Codificar variables categóricas para embeddings
# label_encoders = {}
# for col in categorical_cols:
#     le = LabelEncoder()
#     df[col + '_encoded'] = le.fit_transform(df[col])
#     label_encoders[col] = le
#     print(f"\nCardinalidad de {col}: {len(le.classes_)}")

# Actualizar división con columnas codificadas
train_data = df[df['anio'] < 2021]
val_data = df[df['anio'] == 2021]

# ### MEJORA 3: Usar RobustScaler para manejar outliers
scaler = RobustScaler()
scaler.fit(train_data[numerical_cols])

# Aplicar escalado
train_scaled = train_data.copy()
val_scaled = val_data.copy()
train_scaled[numerical_cols] = scaler.transform(train_data[numerical_cols])
val_scaled[numerical_cols] = scaler.transform(val_data[numerical_cols])

# ### MEJORA 4: Función mejorada para crear secuencias con data augmentation
def create_sequences_augmented(data, window_size, target_col, feature_cols, categorical_cols_encoded, 
                              augment=False, noise_level=0.01):
    """
    Crea secuencias de ventanas deslizantes para el modelo GRU con opción de augmentation.
    """
    sequences_num = []
    sequences_cat = []
    targets = []
    barrio_ids = []
    
    # Agrupar por barrio para mantener continuidad temporal
    for barrio in data['id_bar'].unique():
        barrio_data = data[data['id_bar'] == barrio].sort_index()
        
        if len(barrio_data) <= window_size:
            continue
            
        for i in range(len(barrio_data) - window_size):
            # Secuencia de características numéricas
            seq_num = barrio_data[feature_cols].iloc[i:i+window_size].values
            
            # Data augmentation: agregar ruido gaussiano pequeño durante entrenamiento
            if augment and np.random.random() > 0.5:
                noise = np.random.normal(0, noise_level, seq_num.shape)
                seq_num = seq_num + noise
            
            # Secuencia de características categóricas
            seq_cat = barrio_data[categorical_cols_encoded].iloc[i:i+window_size].values
            # Valor objetivo
            target = barrio_data[target_col].iloc[i+window_size]
            
            sequences_num.append(seq_num)
            sequences_cat.append(seq_cat)
            targets.append(target)
            barrio_ids.append(barrio)
    
    return (np.array(sequences_num, dtype=np.float32), 
            np.array(sequences_cat, dtype=np.int64),
            np.array(targets, dtype=np.float32),
            np.array(barrio_ids))

# ### 4. Definición del Modelo GRU Mejorado con PyTorch

class DengueDataset(Dataset):
    """Dataset personalizado para las secuencias de dengue"""
    def __init__(self, sequences_num, sequences_cat, targets):
        self.sequences_num = sequences_num
        self.sequences_cat = sequences_cat
        self.targets = targets
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.sequences_num[idx]),
                torch.LongTensor(self.sequences_cat[idx]),
                torch.FloatTensor([self.targets[idx]]))

# ### MEJORA 5: Modelo GRU mejorado con Batch Normalization y Attention
class ImprovedGRUModel(nn.Module):
    """
    Modelo GRU mejorado con embeddings, batch normalization, y mecanismo de atención
    """
    def __init__(self, num_features, embedding_dims, hidden_size, num_layers, dropout_rate):
        super(ImprovedGRUModel, self).__init__()
        
        # Embeddings para variables categóricas
        self.embeddings = nn.ModuleList()
        total_embedding_size = 0
        
        # for card, dim in embedding_dims:
        #     self.embeddings.append(nn.Embedding(card, dim))
        #     total_embedding_size += dim
        
        # Tamaño de entrada para GRU
        input_size = num_features + total_embedding_size
        
        # Batch normalization para entrada
        self.input_bn = nn.BatchNorm1d(input_size)
        
        # Capas GRU bidireccionales
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                         batch_first=True, dropout=dropout_rate if num_layers > 1 else 0,
                         bidirectional=True)
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        
        # Batch normalization para hidden states
        self.hidden_bn = nn.BatchNorm1d(hidden_size * 2)
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)
        
        # Capas de salida con residual connection
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x_num, x_cat):
        batch_size, seq_len, _ = x_num.shape
        
        # Procesar embeddings categóricos
        embeddings_list = []
        for i, embedding in enumerate(self.embeddings):
            cat_embedded = embedding(x_cat[:, :, i])
            embeddings_list.append(cat_embedded)
        
        # Concatenar embeddings
        cat_embedded = torch.cat(embeddings_list, dim=-1)
        
        # Concatenar características numéricas y categóricas
        x = torch.cat([x_num, cat_embedded], dim=-1)
        
        # Batch normalization (reshape para BN)
        x = x.view(-1, x.size(-1))
        x = self.input_bn(x)
        x = x.view(batch_size, seq_len, -1)
        
        # Pasar por GRU bidireccional
        out, _ = self.gru(x)
        
        # Attention mechanism
        attention_weights = self.attention(out)
        attention_weights = F.softmax(attention_weights, dim=1)
        
        # Aplicar attention weights
        out = torch.sum(out * attention_weights, dim=1)
        
        # Batch normalization para hidden states
        out = self.hidden_bn(out)
        out = self.dropout(out)
        
        # Capas fully connected con activación
        out = F.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

# ### MEJORA 6: Función de pérdida Huber para robustez ante outliers
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(HuberLoss, self).__init__()
        self.delta = delta
        
    def forward(self, y_pred, y_true):
        residual = torch.abs(y_pred - y_true)
        condition = residual < self.delta
        small_error = 0.5 * torch.square(residual)
        large_error = self.delta * residual - 0.5 * self.delta * self.delta
        return torch.mean(torch.where(condition, small_error, large_error))

# ### 5. Optimización de Hiperparámetros Mejorada con Optuna

def objective(trial):
    """
    Función objetivo mejorada para Optuna
    """
    # Espacio de búsqueda ampliado de hiperparámetros
    window_size = trial.suggest_int('window_size', 12, 48)  # Ventanas más grandes
    hidden_size = trial.suggest_categorical('hidden_size', [16, 32, 64, 128, 256])
    num_layers = trial.suggest_int('num_layers', 2, 4)  # Más capas
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 5e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    
    # Crear secuencias con augmentation
    categorical_cols_encoded = [col + '_encoded' for col in categorical_cols]
    
    X_train_num, X_train_cat, y_train, _ = create_sequences_augmented(
        train_scaled, window_size, target_col_transformed, numerical_cols, 
        categorical_cols_encoded, augment=True
    )
    X_val_num, X_val_cat, y_val, _ = create_sequences_augmented(
        val_scaled, window_size, target_col_transformed, numerical_cols, 
        categorical_cols_encoded, augment=False
    )
    
    # Crear datasets y dataloaders
    train_dataset = DengueDataset(X_train_num, X_train_cat, y_train)
    val_dataset = DengueDataset(X_val_num, X_val_cat, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Configurar modelo
    num_features = len(numerical_cols)
    embedding_dims = []
    # embedding_dims = [
    #     (len(label_encoders['id_bar'].classes_), min(50, len(label_encoders['id_bar'].classes_)//2)),
    #     (len(label_encoders['ESTRATO'].classes_), min(10, len(label_encoders['ESTRATO'].classes_)//2))
    # ]
    
    model = ImprovedGRUModel(num_features, embedding_dims, hidden_size, num_layers, dropout_rate).to(device)
    criterion = HuberLoss(delta=1.0)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
    
    # Entrenamiento con gradient clipping
    n_epochs = 50
    best_val_loss = float('inf')
    patience = 25
    patience_counter = 0
    
    for epoch in range(n_epochs):
        # Training
        model.train()
        train_loss = 0
        for x_num, x_cat, y in train_loader:
            x_num, x_cat, y = x_num.to(device), x_cat.to(device), y.to(device)
            
            optimizer.zero_grad()
            output = model(x_num, x_cat)
            loss = criterion(output, y)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_num, x_cat, y in val_loader:
                x_num, x_cat, y = x_num.to(device), x_cat.to(device), y.to(device)
                output = model(x_num, x_cat)
                loss = criterion(output, y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            break
    
    return best_val_loss

# Ejecutar optimización con Optuna
print("\n### Iniciando optimización de hiperparámetros con Optuna ###")
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\nMejores hiperparámetros encontrados:")
print(study.best_params)
print(f"\nMejor pérdida en validación: {study.best_value:.4f}")

# ### 6. Entrenamiento del Modelo Final con Ensemble

# Obtener mejores hiperparámetros
best_params = study.best_params
window_size = best_params['window_size']
hidden_size = best_params['hidden_size']
num_layers = best_params['num_layers']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']
weight_decay = best_params['weight_decay']

print("\n### Entrenando ensemble de modelos finales ###")

# Combinar datos de entrenamiento y validación
combined_data = pd.concat([train_data, val_data])
combined_scaled = combined_data.copy()
combined_scaled[numerical_cols] = scaler.transform(combined_data[numerical_cols])

# Crear secuencias con datos combinados
categorical_cols_encoded = [col + '_encoded' for col in categorical_cols]

# ### MEJORA 7: Entrenar ensemble de modelos
n_models = 5
models_ensemble = []

for model_idx in range(n_models):
    print(f"\nEntrenando modelo {model_idx + 1}/{n_models}")
    
    # Crear secuencias con diferente semilla para augmentation
    np.random.seed(42 + model_idx)
    X_combined_num, X_combined_cat, y_combined, _ = create_sequences_augmented(
        combined_scaled, window_size, target_col_transformed, numerical_cols, 
        categorical_cols_encoded, augment=True
    )
    
    # Crear dataset y dataloader
    combined_dataset = DengueDataset(X_combined_num, X_combined_cat, y_combined)
    combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
    
    # Configurar modelo
    num_features = len(numerical_cols)
    embedding_dims = [
        (len(label_encoders['id_bar'].classes_), min(50, len(label_encoders['id_bar'].classes_)//2)),
        (len(label_encoders['ESTRATO'].classes_), min(10, len(label_encoders['ESTRATO'].classes_)//2))
    ]
    
    model = ImprovedGRUModel(num_features, embedding_dims, hidden_size, num_layers, dropout_rate).to(device)
    criterion = HuberLoss(delta=1.0)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    
    # Entrenamiento
    n_epochs = 50
    train_losses = []
    
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0
        
        for x_num, x_cat, y in combined_loader:
            x_num, x_cat, y = x_num.to(device), x_cat.to(device), y.to(device)
            
            optimizer.zero_grad()
            output = model(x_num, x_cat)
            loss = criterion(output, y)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(combined_loader)
        train_losses.append(avg_loss)
        scheduler.step(avg_loss)
        
        if (epoch + 1) % 50 == 0:
            print(f"Época {epoch+1}/{n_epochs}, Pérdida: {avg_loss:.4f}")
    
    models_ensemble.append(model)

# ### 7. Generación de Pronósticos Mejorados para 2022

print("\n### Generando pronósticos con ensemble para 2022 ###")

# Preparar datos para pronóstico
all_data = df.copy()
all_data_scaled = all_data.copy()
all_data_scaled[numerical_cols] = scaler.transform(all_data[numerical_cols])

# ### MEJORA 8: Función mejorada para generar pronósticos con ensemble
def generate_ensemble_forecasts(models, data, window_size, n_weeks_ahead, year):
    """
    Genera pronósticos autorregresivos usando ensemble de modelos
    """
    all_forecasts = []
    
    for model in models:
        model.eval()
        model_forecasts = []
        
        # Para cada barrio
        for barrio in data['id_bar'].unique():
            barrio_data = data[data['id_bar'] == barrio].sort_index()
            barrio_id = barrio_data['id_bar'].iloc[0]
            estrato_encoded = barrio_data['ESTRATO_encoded'].iloc[0]
            
            # Obtener última secuencia conocida
            last_sequence_num = barrio_data[numerical_cols].iloc[-window_size:].values
            last_sequence_cat = np.array([[barrio_data['id_bar_encoded'].iloc[0], 
                                          estrato_encoded] for _ in range(window_size)])
            
            # Generar pronósticos semana a semana
            for week in range(1, n_weeks_ahead + 1):
                # Preparar entrada
                x_num = torch.FloatTensor(last_sequence_num).unsqueeze(0).to(device)
                x_cat = torch.LongTensor(last_sequence_cat).unsqueeze(0).to(device)
                
                # Hacer predicción
                with torch.no_grad():
                    pred_scaled = model(x_num, x_cat).cpu().numpy()[0, 0]
                
                # Desescalar predicción y aplicar transformación inversa
                dengue_log_idx = numerical_cols.index(target_col_transformed)
                pred_log_original = pred_scaled * scaler.scale_[dengue_log_idx] + scaler.center_[dengue_log_idx]
                pred_original_scale = np.expm1(pred_log_original)  # Inversa de log1p
                
                # Asegurar que la predicción no sea negativa
                pred_original_scale = max(0, pred_original_scale)
                
                # Guardar predicción
                model_forecasts.append({
                    'id_bar': barrio_id,
                    'anio': year,
                    'semana': week,
                    'dengue': pred_original_scale
                })
                
                # Actualizar secuencia para próxima predicción
                new_row = last_sequence_num[-1].copy()
                new_row[dengue_log_idx] = pred_scaled
                
                # Actualizar features de lag
                dengue_idx = numerical_cols.index('dengue')
                new_row[dengue_idx] = pred_original_scale
                
                # Actualizar ventana deslizante
                last_sequence_num = np.vstack([last_sequence_num[1:], new_row])
        
        all_forecasts.append(pd.DataFrame(model_forecasts))
    
    # Combinar predicciones del ensemble (promedio)
    ensemble_df = pd.concat(all_forecasts)
    ensemble_forecasts = ensemble_df.groupby(['id_bar', 'anio', 'semana'])['dengue'].mean().reset_index()
    
    return ensemble_forecasts

# Generar pronósticos del ensemble
forecasts_2022 = generate_ensemble_forecasts(models_ensemble, all_data_scaled, window_size, 52, 2022)

print(f"\nPronósticos generados: {len(forecasts_2022)} registros")
print("\nEstadísticas de pronósticos:")
print(forecasts_2022['dengue'].describe())

# ### 8. Creación del Archivo de Salida

# Crear DataFrame con formato requerido
output_df = pd.DataFrame()
output_df['id'] = (forecasts_2022['id_bar'].astype(str) + '_' + 
                   forecasts_2022['anio'].astype(str) + '_' + 
                   forecasts_2022['semana'].apply(lambda x: f"{x:02d}"))
output_df['dengue'] = forecasts_2022['dengue'].round(2)

# Verificar formato
print("\nPrimeras filas del archivo de salida:")
print(output_df.head(10))
print(f"\nTotal de registros: {len(output_df)}")

# Guardar archivo CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'pronosticos_GRU_2_optimizado_{timestamp}.csv'
output_df.to_csv(output_filename, index=False)
print(f"\nArchivo '{output_filename}' guardado exitosamente.")

# ### 9. Visualización y Análisis de Resultados

# Visualización de algunos pronósticos
sample_barrios = forecasts_2022['id_bar'].unique()[:5]
fig = go.Figure()

for barrio in sample_barrios:
    barrio_forecast = forecasts_2022[forecasts_2022['id_bar'] == barrio]
    fig.add_trace(go.Scatter(
        x=barrio_forecast['semana'],
        y=barrio_forecast['dengue'],
        mode='lines+markers',
        name=f'Barrio {barrio}',
        line=dict(width=2),
        marker=dict(size=6)
    ))

fig.update_layout(
    title='Pronósticos de Dengue para 2022 - Modelo Optimizado (Muestra de 5 Barrios)',
    xaxis_title='Semana del Año',
    yaxis_title='Casos de Dengue',
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

# Comparación de distribuciones
fig_dist = go.Figure()
fig_dist.add_trace(go.Histogram(x=forecasts_2022['dengue'], name='Pronósticos 2022', nbinsx=50))
fig_dist.update_layout(
    title='Distribución de Pronósticos de Dengue 2022',
    xaxis_title='Casos de Dengue',
    yaxis_title='Frecuencia',
    template='plotly_white'
)
fig_dist.show()

print("\n### Proceso completado exitosamente ###")
print("\nMejoras implementadas para minimizar MSE:")
print("1. Transformación logarítmica de la variable objetivo")
print("2. Ingeniería de características: lags, medias móviles, características cíclicas")
print("3. RobustScaler para manejar outliers")
print("4. Data augmentation con ruido gaussiano")
print("5. Modelo GRU bidireccional con attention mechanism y batch normalization")
print("6. Función de pérdida Huber (robusta ante outliers)")
print("7. Ensemble de 5 modelos")
print("8. Learning rate scheduling y gradient clipping")
print("9. Optimización AdamW con weight decay")
print("10. Mayor espacio de búsqueda de hiperparámetros")

[I 2025-06-26 22:54:08,787] A new study created in memory with name: no-name-5f40fdd6-dcbf-4c60-9612-d7539f91b419



Tamaño conjunto entrenamiento: 3120
Tamaño conjunto validación: 520

Columnas categóricas: ['id_bar', 'ESTRATO']
Columnas numéricas: ['dengue', 'lluvia_mean', 'temperatura_mean']
Variable objetivo: dengue

### Iniciando optimización de hiperparámetros con Optuna ###


  0%|          | 0/50 [00:00<?, ?it/s]

[W 2025-06-26 22:54:08,793] Trial 0 failed with parameters: {'window_size': 25, 'hidden_size': 16, 'num_layers': 2, 'dropout_rate': 0.45985284373248053, 'learning_rate': 0.0010502105436744284, 'batch_size': 64, 'weight_decay': 2.6587543983272695e-05} because of the following error: KeyError("None of [Index(['id_bar_encoded', 'ESTRATO_encoded'], dtype='object')] are in the [columns]").
Traceback (most recent call last):
  File "c:\Git\ICESI\FA_TALLER2\venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\usuario\AppData\Local\Temp\ipykernel_18248\1433406722.py", line 251, in objective
    X_train_num, X_train_cat, y_train, _ = create_sequences_augmented(
                                           ~~~~~~~~~~~~~~~~~~~~~~~~~~^
        train_scaled, window_size, target_col_transformed, numerical_cols,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        categorical_cols_encoded, augment=Tru

KeyError: "None of [Index(['id_bar_encoded', 'ESTRATO_encoded'], dtype='object')] are in the [columns]"